In [165]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime
from dateutil.relativedelta import relativedelta

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

from newspaper import Article

In [166]:
# Driver setting
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [171]:
# 원하는 횟수만큼 Scroll down

def scrollDown(driver, num: int): 
    
    for _ in tqdm(range(0, num), desc='Scrolling Down...', leave='False'):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(rd.uniform(0.1, 0.3))    

    return

In [169]:
# start_date부터 end_date까지 3개월 단위로 나누기

def getPeriods(start_date, end_date):
    
    periods = []
    while (start_date < end_date):
        if (start_date + relativedelta(months=3) > end_date):
            periods.append([start_date.strftime("%Y.%m.%d"), end_date.strftime("%Y.%m.%d")])

        else:
            periods.append([start_date.strftime("%Y.%m.%d"), (start_date + relativedelta(months=3)).strftime("%Y.%m.%d")])
        start_date += relativedelta(months=3)
    # print(periods)

    return periods

In [176]:
# 언론사 세팅

def setCompany():

    company_names = ["YTN", "JTBC", "MBC", "SBS", "국민일보", "매일경제", "조선일보", "중앙일보", "아시아경제", "한국경제", "KBS", "한겨례", "경향신문"]
    company_codes = {
        "YTN": "1052",
        "JTBC": "1437",
        "MBC": "1214",
        "SBS": "1055",
        "국민일보": "1005",
        "매일경제": "1009",
        "조선일보": "1023",
        "중앙일보": "1025",
        "아시아경제": "1277",
        "한국경제": "1015",
        "KBS": "1056",
        "한겨례": "1028",
        "경향신문": "1032",
    }

    return company_names, company_codes

In [173]:
# 기간 세팅 (3개월 단위로 나누기)

start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
periods = getPeriods(start_date, end_date)

periods[0][0] = "2020.09.02"
print(periods)

[['2020.09.02', '2020.12.01'], ['2020.12.01', '2021.03.01'], ['2021.03.01', '2021.06.01'], ['2021.06.01', '2021.09.01'], ['2021.09.01', '2021.12.01'], ['2021.12.01', '2022.03.01'], ['2022.03.01', '2022.06.01'], ['2022.06.01', '2022.09.01'], ['2022.09.01', '2022.12.01'], ['2022.12.01', '2023.03.01'], ['2023.03.01', '2023.06.01'], ['2023.06.01', '2023.09.01'], ['2023.09.01', '2023.12.01'], ['2023.12.01', '2023.12.31']]


In [197]:
driver = setWebdriver()

In [136]:
company_code = companies[companies_name[0]]
query = querys[0]
start_date = periods[0][0]    # 시작 날짜
end_date = periods[0][1]    # 끝 날짜

In [137]:
url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'
print(url)

https://search.naver.com/search.naver?where=news&query=국민의힘&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds=2020.09.02&de=2020.12.01&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1052&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0


In [198]:
driver.get(url)

In [199]:
# 아래로 스크롤 할 수 있을 만큼 스크롤 다운
scrollDown(driver, 99)

Scrolling Down...: 100%|██████████| 99/99 [00:21<00:00,  4.66it/s]


In [200]:
news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

In [201]:
print(len(news_list))

1000


In [203]:
news_url = news_list[0].get_attribute('href')
print(news_url)

https://www.ytn.co.kr/_ln/0101_202011301050011782


In [204]:
article = Article(news_url, language="ko")

In [ ]:
current_url = news_url
article = Article(current_url, language="ko")
article.download()
article.parse()

date = article.publish_date;    print(f'date: {date}')
authors = article.authors;      print(f'authors: {authors}')
title = article.title;  print(f'title: {title}')
text = article.text;    print(f'text: {text}')
print(f'{"-"*200}')

# dates.append(date)
# titles.append(title)
# texts.append(text)

In [191]:
def main():
    
    # 언론사 세팅
    company_names, comapny_codes = setCompany()

    # 기간 세팅 (3개월 단위로 나누기)
    start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
    end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
    periods = getPeriods(start_date, end_date)

    # 검색어 세팅
    querys = ["국민의힘", "더불어민주당"]
    
    
    driver = setWebdriver()

    for comany in tqdm(company_names, desc="Companies..."):
        company_code = comapny_codes[comany]

        for period in periods:
            start_date = period[0]
            end_date = period[1]

            for query in querys:
                url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'

                driver.get(url)
                time.sleep(1)

                scrollDown(driver, 99)  # 아래로 스크롤 할 수 있을 만큼 스크롤 다운

                news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

                for news in news_list:
                    news_url = news.get_attribute('herf')

                    driver.execute_script(f"window.open('{news_url}');")     # 새 탭 열기
                    driver.switch_to.window(driver.window_handles[1])       # 새 탭으로 이동

                    article = Article(current_url, language="ko")
                    article.download()
                    article.parse()

                    date = article.publish_date;    print(f'date: {date}')
                    authors = article.authors;      print(f'authors: {authors}')
                    title = article.title;  print(f'title: {title}')
                    text = article.text;    print(f'text: {text}')
                    print(f'{"-"*200}')
                    
                    time.sleep(1)
                    driver.close()                                          # 현재 탭 닫기
                    driver.switch_to.window(driver.window_handles[0])       # 0번째 탭으로 이동

In [ ]:
if __name__ == "__main__":
    main()

In [195]:
# 첫 번째 탭으로 이동
driver.switch_to.window(driver.window_handles[0])

news_list = driver.find_elements(By.CLASS_NAME, 'news_tit')
print(len(news_list))

0


In [206]:
df = pd.DataFrame(columns=["Date", "Title", "Text"])
df

,Date,Title,Text


In [38]:
df = pd.read_csv('../data/조선일보.csv')
df 

,Unnamed: 0,Query,Date,Title,Text
0,0,국민의힘,2020-12-01,"안철수, 국민의힘 1인 시위현장 방문 “힘 보태겠다”",NaN
1,1,국민의힘,2020-12-01,국민의힘 “야당 안만나고 축구나 하는 최재성 해임하라”,NaN
2,2,국민의힘,2020-12-01,국민 60%가 “추미애·윤석열 국정조사 필요”,NaN
3,3,국민의힘,2020-12-01,"국민의힘 “親조국 삼총사, 법사위에 뭉쳤다. 활약상 상상돼”",NaN
4,4,국민의힘,2020-11-29,"野의원 “국정원, 간첩 대신 국민 부동산 등 사생활 캘수도”",NaN
...,...,...,...,...,...
275,275,더불어민주당,2023-12-28,"윤재옥 “민주당 총선 인재 박선원, 천안함 결함설 주장”",NaN
276,276,더불어민주당,2023-12-29,민주당 “독도 영토분쟁? 尹이 승인했는지 국정조사하자”,NaN
277,277,더불어민주당,2023-12-23,"與 “송영길 내외 부창부수…민주당, 불법행위 드러나면 검찰 탄압 주장”",NaN
278,278,더불어민주당,2023-12-27,이재명 “한동훈 취임 축하… 민주당도 협조할 것은 하겠다”,NaN


In [28]:
df.isnull().sum()

Unnamed: 0     0
Query          0
Date          14
Title         14
Text          20
dtype: int64

In [5]:
import pandas as pd
tmp1 = pd.DataFrame(columns=["Query", "Date", "Title", "Text"])

In [18]:
tmp3 = pd.DataFrame({"Query": "123", "Date": "12", "Title": '123', "Text": '123'}, index=([len(tmp)]))
tmp = pd.concat([tmp, tmp3])   


In [19]:
tmp

,Query,Date,Title,Text
0,123,12,123,123
0,123,12,123,123
2,123,12,123,123
3,123,12,123,123


### Data 확인

In [2]:
import pandas as pd
import numpy as np
import os

In [22]:
datasets = os.listdir('../data/')
print(datasets)

['AsiaGyeongJe.pkl', 'GukMinIlBo.pkl', 'GyeongHyang.pkl', 'HanGukGyeongJe.pkl', 'HanGyeoRye.pkl', 'JoSeonIlBo.pkl', 'JTBC.pkl', 'JungAngIlBo.pkl', 'KBS.pkl', 'MaeIlGyeongJe.pkl', 'MBC.pkl', 'SBS.pkl', 'YTN.pkl']


In [62]:
for dataset in datasets:
    print(f"------------------------------------------------------------------- {dataset} -------------------------------------------------------------------")
    df = pd.read_pickle(f'../data/{dataset}')
    # df = df.drop(["Unnamed: 0", "Date"], axis=1)   # df.drop 설명: 
    print(df.columns)
    # print(f'shape: {df.shape}')
    # print(f'null:\n{df.isnull().sum()}')
    print(df)
    df.to_pickle(f"../data/{dataset.replace('.csv','')}")

------------------------------------------------------------------- AsiaGyeongJe.pkl -------------------------------------------------------------------
Index(['Query', 'Title', 'Text', 'News'], dtype='object')
      Query                                     Title  \
0      국민의힘               국민 10명 중 6명 "秋·尹 국정조사 필요하다"   
1      국민의힘               국민의힘, 혁신노동특위 구성 완료…한국노총도 참여   
2      국민의힘  [아경 여론조사]서울 지지도, 민주당 34.9% vs 국민의힘 24.7%   
3      국민의힘                        [포토] 국민의힘, 법사위 보이콧   
4      국민의힘        국민의힘, '尹 직무배제 부당' 감찰위 결정에 "상식에 부합"   
..      ...                                       ...   
275  더불어민주당             [포토] 한동훈 비대위원장, 이재명 민주당 대표 예방   
276  더불어민주당            [포토]정세균 전 총리와 회동 나선 이재명 민주당 대표   
277  더불어민주당         검찰, 민주당 부대변인 압수수색… 김용 재판 위증 관여 의혹   
278  더불어민주당           대법, '10억대 금품수수' 이정근 징역 4년2개월 확정   
279  더불어민주당         민주당 또 여성비하?…"국힘은 불임정당" 썼다 삭제한 민형배   

                                                  Text   News  
0    [아시아경제 강나훔 기자] 국민 10명 중 6명은 추미애 법무부

In [5]:
import pandas as pd

df.drop('Unnamed: 0', axis=1, inplace=True)


In [42]:
def addCompany(filename: str, company: str):
    df = pd.read_pickle(f'../data/{filename}.pkl')
    df['News'] = company
    df.to_pickle(f'../data/{filename}.pkl')
    print(df)
    
    return

In [60]:
filenames = [file.replace(".pkl", '') for file in os.listdir('../data/')]
print(filenames)
print(len(filenames))

['AsiaGyeongJe', 'GukMinIlBo', 'GyeongHyang', 'HanGukGyeongJe', 'HanGyeoRye', 'JoSeonIlBo', 'JTBC', 'JungAngIlBo', 'KBS', 'MaeIlGyeongJe', 'MBC', 'SBS', 'YTN']
13


In [59]:
companies = ["아시아경제", "국민일보", "경향신문", "한국경제", "한겨례", "조선일보", "JTBC", "중앙일보", "KBS", "매일경제", "MBC", "SBS", "YTN"]
print(len(companies))

13


In [61]:
for i in range(len(filenames)):
    addCompany(filenames[i], companies[i])


      Query                                     Title  \
0      국민의힘               국민 10명 중 6명 "秋·尹 국정조사 필요하다"   
1      국민의힘               국민의힘, 혁신노동특위 구성 완료…한국노총도 참여   
2      국민의힘  [아경 여론조사]서울 지지도, 민주당 34.9% vs 국민의힘 24.7%   
3      국민의힘                        [포토] 국민의힘, 법사위 보이콧   
4      국민의힘        국민의힘, '尹 직무배제 부당' 감찰위 결정에 "상식에 부합"   
..      ...                                       ...   
275  더불어민주당             [포토] 한동훈 비대위원장, 이재명 민주당 대표 예방   
276  더불어민주당            [포토]정세균 전 총리와 회동 나선 이재명 민주당 대표   
277  더불어민주당         검찰, 민주당 부대변인 압수수색… 김용 재판 위증 관여 의혹   
278  더불어민주당           대법, '10억대 금품수수' 이정근 징역 4년2개월 확정   
279  더불어민주당         민주당 또 여성비하?…"국힘은 불임정당" 썼다 삭제한 민형배   

                                                  Text   News  
0    [아시아경제 강나훔 기자] 국민 10명 중 6명은 추미애 법무부 장관과 윤석열 검찰...  아시아경제  
1    [아시아경제 김혜민 기자] 노동 현안을 논의할 국민의힘 노동혁신특별위원회에 한국노총...  아시아경제  
2    아시아경제 의뢰, '정당 지지도 조사' 결과\n\n전체 민주당 34.7% > 국민의...  아시아경제  
3    국민의힘 법사위원들이 30일 국회에서 열린 법사위 전체회의에 불참하고 있다. 이날 ...  아시아

In [57]:
print(len(filenames))

13
